In [ ]:
import os
import torch
import torchvision.utils as vutils
from torch.utils.data import DataLoader
from torch import nn, optim

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 设定输出路径
output_dir = 'generated_images/3'


In [ ]:
# 定义数据加载器

from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms

class FloorPlanDataset(Dataset):
    def __init__(self, black_images_dir, color_images_dir, transform=None):
        self.black_images_dir = black_images_dir
        self.color_images_dir = color_images_dir
        self.transform = transform
        self.black_images = sorted(os.listdir(black_images_dir))
        self.color_images = sorted(os.listdir(color_images_dir))
    
    def __len__(self):
        return len(self.black_images)
    
    def __getitem__(self, idx):
        black_image_path = os.path.join(self.black_images_dir, self.black_images[idx])
        color_image_path = os.path.join(self.color_images_dir, self.color_images[idx])
        
        black_image = Image.open(black_image_path).convert('RGB')
        color_image = Image.open(color_image_path).convert('RGB')
        
        if self.transform:
            black_image = self.transform(black_image)
            color_image = self.transform(color_image)
        
        return black_image, color_image

# 定义数据变换
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # 调整图像大小
    transforms.ToTensor()           # 转换为张量
])

# 初始化数据集和数据加载器
black_images_dir = '../Dataset/A'  # 黑色轮廓图所在的目录
color_images_dir = '../Dataset/B'  # 彩色照片所在的目录

dataset = FloorPlanDataset(black_images_dir, color_images_dir, transform)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, num_workers=0)


In [ ]:
# 优化生成器
import torch
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, input_nc, output_nc, ngf=64):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # 输入是 (input_nc) x 256 x 256
            nn.Conv2d(input_nc, ngf, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # 状态大小： (ngf) x 128 x 128
            nn.Conv2d(ngf, ngf * 2, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # 状态大小： (ngf * 2) x 64 x 64
            nn.Conv2d(ngf * 2, ngf * 4, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # 状态大小： (ngf * 4) x 32 x 32
            nn.Conv2d(ngf * 4, ngf * 8, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # 状态大小： (ngf * 8) x 16 x 16
            nn.Conv2d(ngf * 8, ngf * 8, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # 状态大小： (ngf * 8) x 8 x 8
            nn.ConvTranspose2d(ngf * 8, ngf * 8, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # 状态大小： (ngf * 8) x 16 x 16
            nn.ConvTranspose2d(ngf * 8, ngf * 4, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # 状态大小： (ngf * 4) x 32 x 32
            nn.ConvTranspose2d(ngf * 4, ngf * 2, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # 状态大小： (ngf * 2) x 64 x 64
            nn.ConvTranspose2d(ngf * 2, ngf, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # 状态大小： (ngf) x 128 x 128
            nn.ConvTranspose2d(ngf, output_nc, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
            # 输出大小： (output_nc) x 256 x 256
        )

    def forward(self, input):
        return self.main(input)


In [ ]:
# 优化判别器模型
import torch.nn as nn

class Discriminator(nn.Module):
    def __init__(self, input_nc, ndf=64):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # 输入是 (input_nc) x 256 x 256
            nn.Conv2d(input_nc, ndf, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.25),

            # 状态大小： (ndf) x 128 x 128
            nn.Conv2d(ndf, ndf * 2, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.25),

            # 状态大小： (ndf * 2) x 64 x 64
            nn.Conv2d(ndf * 2, ndf * 4, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.25),

            # 状态大小： (ndf * 4) x 32 x 32
            nn.Conv2d(ndf * 4, ndf * 8, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.25),

            # 状态大小： (ndf * 8) x 16 x 16
            nn.Conv2d(ndf * 8, ndf * 8, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.25),
            
            # 状态大小： (ndf * 8) x 8 x 8
            nn.Conv2d(ndf * 8, 1, kernel_size=4, stride=1, padding=0),
            nn.Sigmoid()
            # 输出大小： 1 x 1 x 1
        )

    def forward(self, input):
        return self.main(input)


In [ ]:
# 模型初始化
# 初始化生成器和判别器
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator = Generator(input_nc=3, output_nc=3).to(device)
discriminator = Discriminator(input_nc=6).to(device)

# 权重初始化
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0.0)

generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)


In [ ]:
# 定义损失函数和优化器
# 定义损失函数
criterion_GAN = nn.BCEWithLogitsLoss()
criterion_L1 = nn.L1Loss()

# 定义优化器
# 定义学习率调度器

batch_size = 8  

optimizer_G = optim.Adam(generator.parameters(), lr=0.0001, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0001, betas=(0.5, 0.999))

# 标签平滑
real_labels = torch.full((batch_size,), 0.9, dtype=torch.float, device=device)
fake_labels = torch.full((batch_size,), 0.0, dtype=torch.float, device=device)



In [ ]:
# 设定输出路径
output_dir = 'generated_images/3'
os.makedirs(output_dir, exist_ok=True)

In [ ]:
num_epochs = 100

for epoch in range(num_epochs):
    for i, (black_images, color_images) in enumerate(dataloader):
        black_images = black_images.to(device)
        color_images = color_images.to(device)

        # ---------------------------
        # 训练判别器
        # ---------------------------
        fake_images = generator(black_images)

        # 拼接黑色框图和彩色图像作为判别器输入
        real_input = torch.cat((black_images, color_images), dim=1)
        fake_input = torch.cat((black_images, fake_images.detach()), dim=1)

        # 将拼接后的输入传递给判别器
        output_real = discriminator(real_input)
        output_fake = discriminator(fake_input)

        real_labels = torch.ones_like(output_real).to(device)
        fake_labels = torch.zeros_like(output_fake).to(device)

        loss_D_real = criterion_GAN(output_real, real_labels)
        loss_D_fake = criterion_GAN(output_fake, fake_labels)
        loss_D = (loss_D_real + loss_D_fake) / 2

        optimizer_D.zero_grad()
        loss_D.backward()
        optimizer_D.step()

        # ---------------------------
        # 训练生成器
        # ---------------------------
        # 再次使用拼接后的生成图像输入到判别器
        fake_input = torch.cat((black_images, fake_images), dim=1)
        output_fake = discriminator(fake_input)
        loss_G_GAN = criterion_GAN(output_fake, real_labels)  # 生成器希望其输出被判别器识别为真实
        loss_G_L1 = criterion_L1(fake_images, color_images)  # 生成的图像与真实图像的 L1 损失
        loss_G = loss_G_GAN + 100 * loss_G_L1

        optimizer_G.zero_grad()
        loss_G.backward()
        optimizer_G.step()

        # 每个批次打印一次损失
        print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(dataloader)}], "
              f"Loss_D: {loss_D.item():.4f}, Loss_G: {loss_G.item():.4f}")

    # ---------------------------
    # 保存图像
    # ---------------------------
    with torch.no_grad():
        # 生成彩色图像
        fake_images = generator(black_images)

        # 拼接黑色框图和生成的彩色图像，沿宽度方向（dim=3）拼接
        combined_images = torch.cat((black_images, fake_images), dim=3)

        # 保存拼接后的图像
        for idx, combined_image in enumerate(combined_images):
            save_path = os.path.join(output_dir, f"epoch_{epoch+1}_image_{idx+1}.png")
            vutils.save_image(combined_image, save_path, normalize=True)

        print(f"Saved combined images in {output_dir} for epoch {epoch+1}")
